In [1]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 
from netCDF4 import default_fillvals
import configparser
import pandas as pd 

# import soil classification functions
from parameter_functions import (is_soil_class, is_param_value, classify_soil_texture)

# import veg functions
from parameter_functions import (calculate_cv_pft, calculate_nveg_pfts, 
                                 map_pft_to_nldas_class, is_overstory, 
                                 calc_root_fract, calc_root_depth_rz1, calc_root_depth_rz2)
# import soil layer aggregation functions 
from parameter_functions import (calculate_first_layer_harmonic_mean, calculate_second_layer_harmonic_mean, 
                                 calculate_third_layer_harmonic_mean, calculate_first_layer_arithmetic_mean, 
                                 calculate_second_layer_arithmetic_mean, calculate_third_layer_arithmetic_mean,
                                 soil_class_values, calculate_init_moist, calculate_baseflow_parameters,
                                 create_empty_arrays, create_parameter_dataset)

# define fillvals
fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

This is a notebook for deriving new VIC 5 parameters in the RASM domain at multiple resolutions. 

Currently 50km (`wr50a_ar9v`) and 25km (`wr50a_ar9v`) are supported. 

The accompanying excel sheet `deriving_new_parameters_v2.xlsx` and Word Doc `procedure_for_derivation.docx` provide additional citations and details on methods. 

__Load configuration file for deriving parameters__

In [2]:
cwd = os.getcwd()
config = configparser.ConfigParser()
config.read(os.path.join(cwd, 'regridding', 'regridding.cfg'))

['/p/home/gergel/scripts/rasm-vic5-parameters/regridding/regridding.cfg']

Set domain file

In [3]:
domain = xr.open_dataset(os.path.join(config['Parameter Specs']['domain_file_dir'], 
                                      config['Parameter Specs']['domain_file']))

masknan_vals = domain['mask'].where(domain['mask'] == 1).values

__Set resolution for filenames__

In [4]:
res = config['Parameter Specs']['res']
grid = config['Parameter Specs']['grid']

nj = len(domain.nj)
ni = len(domain.ni)
num_gridcells = nj * ni
    
num_veg = 17
print("calculating parameters at %s" %res)

calculating parameters at 25km


__Load 50km and global parameters for some derivations__

In [5]:
old_params = xr.open_dataset(os.path.join(config['Other']['dir'], 
                                          config['Other']['old_param_filename']))

__Options__

In [6]:
organic_fract = config.getboolean('Options', 'organic_fract')
max_snow_albedo = config.getboolean('Options', 'max_snow_albedo')
bulk_density_comb = config.getboolean('Options', 'bulk_density_comb')

__Load soil data__

In [7]:
soil_data_vars = collections.OrderedDict()
soil_data_vars['silt'] = 'silt_sl*'
soil_data_vars['sand'] = 'sand_sl*'
soil_data_vars['clay'] = 'clay_sl*'
soil_data_vars['bulk_density'] = 'bulk_density_sl*'
soil_data_vars['organic_fract'] = 'organic_fract_sl*'

# soil data dict with nlayer = 7 (base resolution of data)
soil_data = {}
soil_data_dir = config['Parameter Specs']['output_dir']
for soil_var, soil_wildcard in soil_data_vars.items(): 
    soil_data[soil_var] = xr.open_mfdataset(os.path.join(soil_data_dir, soil_wildcard),
                                            concat_dim='nlayer', 
                                            data_vars='all', 
                                            coords='all')

__Calculate soil types based on percent clay, percent sand and bulk density__


In [8]:
# classify_soil_texture(sand, clay, silt)
soil_type_array = xr.apply_ufunc(classify_soil_texture, 
                                 soil_data['sand']['sand'].where(domain.mask == 1), 
                                 soil_data['clay']['clay'].where(domain.mask == 1),  
                                 soil_data['silt']['silt'].where(domain.mask == 1),
                                 dask='allowed',
                                 vectorize=True)

In [9]:
ksat = xr.apply_ufunc(soil_class_values, 
                      soil_type_array,
                      'ksat',
                      dask='allowed',
                      vectorize=True)
quartz = xr.apply_ufunc(soil_class_values, 
                        soil_type_array,
                        'quartz',
                        dask='allowed',
                        vectorize=True)
Wcr_FRACT = xr.apply_ufunc(soil_class_values, 
                           soil_type_array,
                           'Wcr_FRACT',
                           dask='allowed',
                           vectorize=True)
Wpwp_FRACT = xr.apply_ufunc(soil_class_values, 
                            soil_type_array,
                           'Wpwp_FRACT',
                            dask='allowed',
                            vectorize=True)
b = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'b',
                   dask='allowed',
                   vectorize=True)
bulk_density_min = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'bulk_density',
                   dask='allowed',
                   vectorize=True)
resid_moist = xr.apply_ufunc(soil_class_values, 
                             soil_type_array,
                             'resid_moist',
                             dask='allowed',
                             vectorize=True)

__Load regridded GTOPO 30 data, data var for elevation is called `Band1`, in (m)__

In [10]:
gtopo_filename, gtopo_fileext = os.path.splitext(config['GTOPO']['filename'])
gtopo = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'], 
                                     '%s_%s.nc' %(gtopo_filename, grid)))
elev = gtopo['Band1']

__Load regridded WORLDCLIM climate data for annual t and p__

In [11]:
worldclim_direc = config['Parameter Specs']['output_dir']
prec = xr.open_mfdataset(os.path.join(worldclim_direc, 'prec*'),
                                      concat_dim='time', 
                                      data_vars=['prec'], 
                                      coords='all')

# aggregate to annual, need average annual precip
annual_precip = prec['prec'].sum('time')

temp = xr.open_mfdataset(os.path.join(worldclim_direc, 'tavg*'),
                                      concat_dim='time', 
                                      data_vars='all', 
                                      coords='all')
tavg = temp['tavg'].mean('time')

__Load CLM PFTs to use for vegetation parameters__

In [12]:
pfts_filename, pfts_fileext = os.path.splitext(config['PFTs']['filename'])
veg_data = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'], 
                                        '%s_%s.nc' %(pfts_filename, grid)))

__Calculate Cv from PFTs__

In [13]:
cv = xr.apply_ufunc(calculate_cv_pft, 
                    veg_data['PCT_PFT'].where(domain.mask == 1),
                    dask='allowed',
                    vectorize=True)

__Calculate number of active PFTs, `Nveg`__ 

In [14]:
Nveg = xr.apply_ufunc(calculate_nveg_pfts,
                      veg_data['PCT_PFT'].where(domain.mask == 1),
                      dask='allowed',
                      input_core_dims=[['pft']],
                      vectorize=True)

__Load LAI and vegetation height, `MONTHLY_LAI` and `MONTHLY_HEIGHT_TOP`__

In [15]:
lai_filename, lai_fileext = os.path.splitext(config['Vegetation']['lai_filename'])
lai_file = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'], 
                                   '%s_%s_%s.nc' %(lai_filename, grid, "lai")))

In [16]:
veg_height_filename, veg_height_fileext = os.path.splitext(config['Vegetation']['veg_height_filename'])
veg_height_file = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'], 
                                '%s_%s_%s.nc' %(veg_height_filename, grid, "veg_height")))

LAI and veg_height from CLM and `PCT_PFT` from CLM have a different number of PFTs (`PCT_PFT` has one more PFT, 17 vs 16). The extra PFT in `PCT_PFT` has `PCT_PFT` = 0 over the entire RASM domain, so I just slice the LAI and veg_height from the 0th PFT (water/bare soil) and concatenate it for the 16th PFT. 

In [17]:
lai_slice = lai_file['MONTHLY_LAI'].isel(pft = 0)
vegheight_slice = veg_height_file['MONTHLY_HEIGHT_TOP'].isel(pft=0)

In [18]:
lai = xr.concat([lai_file['MONTHLY_LAI'], lai_slice], dim='pft')
veg_height = xr.concat([veg_height_file['MONTHLY_HEIGHT_TOP'], vegheight_slice], dim='pft')

In [19]:
veg_rough = 0.123 * veg_height
displacement = 0.67 * veg_height

displacement.values[displacement.values == 0] = 1.0

__Change dims and order of dims of LAI array__

In [20]:
lai = lai.rename({'time': 'month', 'pft': 'veg_class'})
lai = lai.transpose('veg_class', 'month', 'nj', 'ni')

veg_rough = veg_rough.rename({'time': 'month', 'pft': 'veg_class'})
veg_rough = veg_rough.transpose('veg_class', 'month', 'nj', 'ni')

displacement = displacement.rename({'time': 'month', 'pft': 'veg_class'})
displacement = displacement.transpose('veg_class', 'month', 'nj', 'ni')

__Note__: map albedo, root zone fraction and root zone depth based on vegetation type. see `deriving_new_parameters_v2.xlsx` sheet titled `PFT-NLDAS Mapping` for mapping between NLDAS vegetation classes (used in old VIC 5 parameters) and CLM PFTs. This mapping is based on obvious relationships and some approximations (used for PFTs 8-11).

__Create Dataset for variables and define data_vars__

In [21]:
arr_months, arr_nlayer, \
arr_rootzone, arr_veg_classes, \
arr_veg_classes_rootzone, arr_veg_classes_month = create_empty_arrays(domain, nj, ni, num_veg)

In [22]:
# create DataSet
params = create_parameter_dataset(domain, old_params, nj, ni, num_veg, organic_fract, max_snow_albedo, 
                                  bulk_density_comb)

# fill in values
params['Cv'].values = cv.values
params['Nveg'].values = Nveg.values
params['LAI'].values = lai.values.reshape(num_veg, 12, nj, ni)
params['displacement'].values = displacement.values.reshape(num_veg, 12, nj, ni)
params['veg_rough'].values = veg_rough.values.reshape(num_veg, 12, nj, ni)
params['elev'].values = elev.values
params['avg_T'].values = tavg.values
params['annual_prec'].values = annual_precip.values

roughness = np.copy(masknan_vals)
roughness[np.nonzero(masknan_vals)] = 0.001
params['rough'].values = roughness

define `trunk_ratio`, `rarc`, `rmin`, `wind_h`, `RGL`, `rad_atten`, `wind_atten`, `overstory`, `max_snow_albedo`

In [23]:
# trunk ratio, rarc, rad_atten
trunk_ratio = np.copy(arr_veg_classes)
params['trunk_ratio'].values = trunk_ratio * 0.2
# adjust for bare soil 
params['trunk_ratio'].values[0, :, :] = 0.0

rarc = np.copy(arr_veg_classes)
params['rarc'].values = rarc * 60
# adjust for bare soil
params['rarc'].values[0, :, :] = 100

rad_atten = np.copy(arr_veg_classes)
params['rad_atten'].values = rad_atten * 0.5
# adjust for bare soil 
params['rad_atten'].values[0, :, :] = 0.0

wind_atten = np.copy(arr_veg_classes)
params['wind_atten'].values = wind_atten * 0.5
# adjust for bare soil 
params['wind_atten'].values[0, :, :] = 0.0

In [24]:
if max_snow_albedo == True:
    # max_albedo
    for pft in veg_data.pft.values:
        # get nldas mapping from pft
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0:
            max_alb = 0.34
        elif nldas == 1:
            max_alb = 0.37
        elif nldas == 2:
            max_alb = 0.35
        elif nldas == 3: 
            max_alb = 0.35
        elif nldas == 4: 
            max_alb = 0.44
        elif nldas == 5:
            max_alb = 0.69
        elif nldas == 6:
            max_alb = 0.43
        elif nldas == 7:
            max_alb = 0.56
        elif nldas == 8:
            max_alb = 0.70
        elif nldas == 9:
            max_alb = 0.65
        elif nldas == 10:
            max_alb = 0.46
        elif nldas == 11:
            max_alb = 0.84
        params['max_snow_albedo'].values[pft, :, :] = np.ones((1, nj, ni)) * max_alb

In [25]:
# rmin, wind_h
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=0).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas == 4:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=4).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 5 and nldas <= 6:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=5).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=5).mean())
    elif nldas >= 7 and nldas <= 8:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=7).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=7).mean())
    elif nldas == 9:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=9).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 10:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=10).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=10).mean())
    elif nldas == 11:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=11).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['rmin'].values[pft, :, :] = np.ones((1, nj, ni)) * rmin
    params['wind_h'].values[pft, :, :] = np.ones((1, nj, ni)) * wind_h

In [26]:
# RGL
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas >= 4 and nldas <= 5:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 6 and nldas <= 8:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=6).mean())
    elif nldas >= 9 and nldas <= 10:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 11:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['RGL'].values[pft, :, :] = np.ones((1, nj, ni)) * rgl

In [27]:
# overstory
overstory = np.copy(arr_veg_classes)
for pft in veg_data.pft.values:
    nldas = map_pft_to_nldas_class(pft)
    if nldas > 6:
        # no overstory
        overstory = 0.0
    else: 
        overstory = 1.0
    params['overstory'].values[pft, :, :] = overstory

In [28]:
root_depth_rz1 = xr.apply_ufunc(calc_root_depth_rz1,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth_rz2 = xr.apply_ufunc(calc_root_depth_rz2,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth = xr.concat([root_depth_rz1, root_depth_rz2],
                      dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [29]:
# root fract 

rz = 0
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz1 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft),
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz1 = xr.concat([root_fract_rz1, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft),
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
rz = 1
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz2 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft), 
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz2 = xr.concat([root_fract_rz2, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft), 
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
        
root_fract = xr.concat([root_fract_rz1, root_fract_rz2], dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [30]:
params['root_depth'].values = root_depth
params['root_fract'].values = root_fract

__albedo__

In [31]:
# loop over pft classes and months 
for pft in veg_data.pft.values:
    for month in old_params.month.values:
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0 or nldas == 1:
            albedo = 0.12
        elif nldas >= 2 and nldas <= 5:
            albedo = 0.18
        elif nldas >= 6 and nldas <= 8:
            albedo = 0.19
        elif nldas == 9:
            albedo = 0.2
        elif nldas == 10: 
            albedo = 0.12
        elif nldas == 11: 
            albedo = 0.2
        params['albedo'].values[pft, month-1, :, :] = np.ones((1, 1, nj, ni)) * albedo

__Load hydroclimate classes__

In [32]:
hydro_classes = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'],
                                             'hydroclimate_masks_%s.nc' %grid))

__Calculate baseflow parameters: Ds, Dsmax, Ws__

In [33]:
soil_direc = config['Soil Data']['ascii_dir']
soil_filename = config['Soil Data']['ascii_filename']
d1 = calculate_baseflow_parameters(domain, soil_direc, soil_filename, hydro_classes, "d1")
params['Ds'].values = d1

d2 = calculate_baseflow_parameters(domain, soil_direc, soil_filename, hydro_classes, "d2")
params['Dsmax'].values = d2

d3 = calculate_baseflow_parameters(domain, soil_direc, soil_filename, hydro_classes, "d3")
params['Ws'].values = d3

d4 = calculate_baseflow_parameters(domain, soil_direc, soil_filename, hydro_classes, "d4")
params['c'].values = d4

__b_i (`infilt`)__

In [34]:
bi = np.copy(masknan_vals)
bi[np.nonzero(hydro_classes['arid'].values)] = 0.05
bi[np.nonzero(hydro_classes['temperate_dry'].values)] = 0.05
bi[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
bi[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.25
bi[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.25
bi[np.nonzero(hydro_classes['polar'].values)] = 0.35

params['infilt'].values = bi

__soil depths (`depth`)__

In [35]:
D1 = np.copy(masknan_vals)
D2 = np.copy(masknan_vals)
D3 = np.copy(masknan_vals)
D1[np.nonzero(domain.mask.values)] = 0.3
D3[np.nonzero(domain.mask.values)] = 0.5

D2[np.nonzero(hydro_classes['arid'].values)] = 2.0
D2[np.nonzero(hydro_classes['temperate_dry'].values)] = 2.0
D2[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 2.0
D2[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 1.1
D2[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.3
D2[np.nonzero(hydro_classes['polar'].values)] = 0.3


depths = np.rollaxis(np.dstack((D1, D2, D3)), axis=2)
params['depth'].values = depths

__Aggregate ISRIC soil data to VIC soil depths__

first need array of soil depths 

In [36]:
soil_depths = params['depth'].sum(axis=0)
print("max soil depth is %.1f m" % soil_depths.max())

max soil depth is 2.8 m


In [37]:
ksat_l1 = xr.apply_ufunc(calculate_first_layer_harmonic_mean,
                         ksat.isel(nlayer=0), 
                         ksat.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

ksat_l2 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_l3 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_vals = np.rollaxis(np.dstack((ksat_l1, ksat_l2, ksat_l3)), 
                        axis=2)

params['Ksat'].values = ksat_vals

In [38]:
# bulk_density
bdm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=0), 
                         bulk_density_min.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

bdm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_vals = np.rollaxis(np.dstack((bdm_l1, bdm_l2, bdm_l3)), 
                        axis=2)
params['bulk_density'].values = bdm_vals

In [39]:
# expt
b_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         b.isel(nlayer=0), 
                         b.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

b_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

b_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

expt_vals = np.rollaxis(np.dstack(((b_l1 * 2) + 3, (b_l2 * 2) + 3, (b_l3 * 2) + 3)), 
                        axis=2)
params['expt'].values = expt_vals
params['bubble'].values = (np.copy(params['expt'].values) * 0.32) + 4.3

In [40]:
# resid_moist
rm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=0), 
                         resid_moist.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

rm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

rm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
rm_vals = np.rollaxis(np.dstack((rm_l1, rm_l2, rm_l3)), 
                        axis=2)
params['resid_moist'].values = rm_vals

In [41]:
# Wcr_FRACT
wcr_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=0), 
                         Wcr_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wcr_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wcr_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
wcr_vals = np.rollaxis(np.dstack((wcr_l1, wcr_l2, wcr_l3)), 
                        axis=2)

In [42]:
# Wpwp_FRACT
wpwp_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=0), 
                         Wpwp_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wpwp_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_vals = np.rollaxis(np.dstack((wpwp_l1, wpwp_l2, wpwp_l3)), 
                        axis=2)

In [43]:
# quartz
qz_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         quartz.isel(nlayer=0), 
                         quartz.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

qz_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

qz_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
qz_vals = np.rollaxis(np.dstack((qz_l1, qz_l2, qz_l3)), 
                        axis=2)
params['quartz'].values = qz_vals

In [44]:
if bulk_density_comb == True:
    # bulk_density
    bd_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=0), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=1),
                             dask='allowed',
                             vectorize=True)

    bd_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)

    bd_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)
    bd_vals = np.rollaxis(np.dstack((bd_l1, bd_l2, bd_l3)), 
                            axis=2)
    params['bulk_density_comb'].values = bd_vals

In [45]:
if organic_fract == True:
    # organic fract
    of_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=0), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=1),
                             dask='allowed',
                             vectorize=True)

    of_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)

    of_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)
    of_vals = np.rollaxis(np.dstack(((of_l1/1000), (of_l2/1000), (of_l3/1000))), 
                            axis=2)
    params['organic'].values = of_vals

__Calculate porosity from bulk density and soil density__

In [46]:
sd_l1 = np.copy(masknan_vals)
sd_l2 = np.copy(masknan_vals)
sd_l3 = np.copy(masknan_vals)

sd_l1[np.nonzero(masknan_vals)] = 2685.0
sd_l2[np.nonzero(masknan_vals)] = 2685.0
sd_l3[np.nonzero(masknan_vals)] = 2685.0

sd_vals = np.rollaxis(np.dstack((sd_l1, sd_l2, sd_l3)), 
                        axis=2)
params['soil_density'].values = sd_vals

In [47]:
if organic_fract == True:
    sd_org_l1 = np.copy(masknan_vals)
    sd_org_l2 = np.copy(masknan_vals)
    sd_org_l3 = np.copy(masknan_vals)

    sd_org_l1[np.nonzero(masknan_vals)] = 1300.0
    sd_org_l2[np.nonzero(masknan_vals)] = 1300.0
    sd_org_l3[np.nonzero(masknan_vals)] = 1300.0

    sd_org_vals = np.rollaxis(np.dstack((sd_org_l1, sd_org_l2, sd_org_l3)), 
                            axis=2)
    params['soil_density_org'].values = sd_org_vals

In [48]:
# calculate porosity
if bulk_density_comb == True:
    porosity = 1 - (params['bulk_density_comb'] / params['soil_density'])
else:
    porosity = 1 - (params['bulk_density'] / params['soil_density'])

In [49]:
params['Wpwp_FRACT'].values = wpwp_vals / porosity.values
params['Wcr_FRACT'].values = wcr_vals / porosity.values

__Make initial moisture fully saturated__

In [50]:
init_moist_l1 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=0), 
                               params.depth.isel(nlayer=0),
                               dask='allowed', 
                               vectorize=True)
init_moist_l2 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=1),
                               dask='allowed', 
                               vectorize=True)
init_moist_l3 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=2),
                               dask='allowed', 
                               vectorize=True)
init_moist_vals = np.rollaxis(np.dstack((init_moist_l1, init_moist_l2, init_moist_l3)), 
                        axis=2)
params['init_moist'].values = init_moist_vals

__Add `off_gmt`__

In [51]:
if res == "50km":
    params['off_gmt'].values = old_params['off_gmt'].values

else:
    # load regridded off_gmt
    gmt_filename, gmt_fileext = os.path.splitext(config['Other']['gmt_regrid_filename'])
    off_gmt = xr.open_dataset(os.path.join(config['Parameter Specs']['output_dir'], 
                                           '%s_%s.nc' %(gmt_filename, grid)))
    params['off_gmt'].values = off_gmt['off_gmt'].values

In [52]:
phi_s = np.copy(arr_nlayer)
phi_s[np.nonzero(arr_nlayer)] = np.asscalar(old_params['phi_s'].mean())
params['phi_s'].values = phi_s

# use domain mask since frozen soils should be True for all gridcells
params['fs_active'].values = domain['mask'].values

dp = np.copy(masknan_vals)
dp[np.nonzero(masknan_vals)] = np.asscalar(old_params['dp'].mean())
params['dp'].values = dp

In [53]:
snow_rough = np.copy(masknan_vals)
snow_rough[np.nonzero(masknan_vals)] = 0.0024
params['snow_rough'].values = snow_rough

__Make gridcell number array__

In [54]:
# add run_cell, mask, xv and yv, xc, yc, gridcell, lats, lons
runcell = np.copy(masknan_vals)
runcell[np.nonzero(masknan_vals)] = 1
params['run_cell'].values = runcell
params['mask'].values = domain['mask'].values

if res == "50km":
    gc_arr = old_params['gridcell'].values
else: 
    gridcell_nums = np.linspace(1, nj*ni, nj*ni, endpoint=True, dtype='int32')
    # gc_arr = np.fliplr(np.flipud(gridcell_nums.reshape(nj, ni)))
    gc_arr = gridcell_nums.reshape(nj, ni)
    
params['gridcell'].values = gc_arr
params['lats'].values = domain['yc'].where(domain.mask==1).values
if res == "50km":
    params['lons'].values = old_params['lons'].where(domain.mask==1).values
else:
    params['lons'].values = domain['xc'].where(domain.mask==1).values
params['xc'].values = domain['xc'].values
params['yc'].values = domain['yc'].values

params['xv'].values = np.rollaxis(domain['xv'].values, axis=2)

swap 0th and 16th veg class to accommodate bare soil. 

veg class vars: Cv, Nveg, trunk_ratio, rarc, rmin, wind_h, RGL, rad_atten, wind_atten, albedo, LAI, overstory, displacement, veg_rough, root_depth, root_fract

additional organic fract parameters if that option is set to True: 

soil particle density of OM 

bulk density of OM

organic content of soil (fraction of total soil volume)

Note: Organic matter (%) = Total organic carbon (%) x 1.72, from http://www.soilquality.org.au/factsheets/organic-carbon

In [55]:
if max_snow_albedo == True:
    veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough', 'max_snow_albedo']
else: 
    veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough']

for veg_class_var in veg_class_vars: 
    if ((veg_class_var == "LAI") or (veg_class_var == "albedo") or (veg_class_var == "root_depth") 
    or (veg_class_var == "root_fract") or (veg_class_var == "veg_rough") or 
        (veg_class_var == "displacement")):
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :, :] = last
        params[veg_class_var].values[16, :, :, :] = bare
    else:
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :] = last
        params[veg_class_var].values[16, :, :] = bare
params['root_fract'].values[16, :, :, :] = 0
params['root_depth'].values[16, :, :, :] = 0
params['displacement'].values[16, :, :, :] = 0
params['veg_rough'].values[16, :, :, :] = 0
params['overstory'].values[16, :, :] = 0

In [56]:
# adjust data vars that need adjusting 
params['run_cell'].values = domain['mask'].where(domain.mask == 1)
params['gridcell'].values = params['gridcell'].where(domain.mask == 1)
params['Ksat'].values = params['Ksat'].where(domain.mask == 1)
params['expt'].values = params['expt'].where(domain.mask == 1)
params['bubble'].values = params['bubble'].where(domain.mask == 1)
params['Wpwp_FRACT'].values = params['Wpwp_FRACT'].where(domain.mask == 1)
params['Wcr_FRACT'].values = params['Wcr_FRACT'].where(domain.mask == 1)
params['resid_moist'].values = params['resid_moist'].where(domain.mask == 1)
params['quartz'].values = params['quartz'].where(domain.mask == 1)
if bulk_density_comb == True:
    params['bulk_density_comb'].values = params['bulk_density_comb'].where(domain.mask == 1)
params['bulk_density'].values = params['bulk_density'].where(domain.mask == 1)
params['soil_density'].values = params['soil_density'].where(domain.mask == 1)
params['c'].values = params['c'].where(domain.mask == 1)
params['dp'].values = params['dp'].where(domain.mask == 1)
params['snow_rough'].values = params['snow_rough'].where(domain.mask == 1)
params['Nveg'].values = params['Nveg'].where(domain.mask == 1)
params['trunk_ratio'] = params['trunk_ratio'].where(domain.mask == 1)
params['rarc'] = params['rarc'].where(domain.mask == 1)
params['phi_s'] = params['phi_s'].where(domain.mask == 1)
params['rmin'] = params['rmin'].where(domain.mask == 1)
params['wind_h'] = params['wind_h'].where(domain.mask == 1)
params['RGL'] = params['RGL'].where(domain.mask == 1)
params['rad_atten'] = params['rad_atten'].where(domain.mask == 1)
params['wind_atten'] = params['wind_atten'].where(domain.mask == 1)
if max_snow_albedo == True:
    params['max_snow_albedo'] = params['max_snow_albedo'].where(domain.mask == 1)
params['root_depth'] = params['root_depth'].where(domain.mask == 1)
params['root_fract'] = params['root_fract'].where(domain.mask == 1)
params['albedo'] = params['albedo'].where(domain.mask == 1)
params['LAI'] = params['LAI'].where(domain.mask == 1)
params['overstory'] = params['overstory'].where(domain.mask == 1)
params['displacement'] = params['displacement'].where(domain.mask == 1)
params['veg_rough'] = params['veg_rough'].where(domain.mask == 1)
params['elev'] = params['elev'].where(domain.mask == 1)
params['avg_T'] = params['avg_T'].where(domain.mask == 1)
params['annual_prec'] = params['annual_prec'].where(domain.mask == 1)
params['rough'] = params['rough'].where(domain.mask == 1)
if organic_fract == True:
    params['organic'] = params['organic'].where(domain.mask == 1)
    params['soil_density_org'] = params['soil_density_org'].where(domain.mask == 1)

In [57]:
max_moist = params['depth'] * porosity * 1000
Wcr = params['Wcr_FRACT'] * max_moist
Wpwp = params['Wpwp_FRACT'] * max_moist
resid_moist_mm = params['resid_moist'] * params['depth'] * 1000

__Some quick tests to ensure that the parameters don't make VIC crash:__

In [58]:
if (Wcr.where(Wpwp > Wcr).sum()) > 0:
    raise AssertionError("wilting point moisture is greater than critical point moisture")

In [59]:
if (params['resid_moist'].where(Wpwp < params['resid_moist']).sum()) > 0:
    raise AssertionError("wilting point moisture is less than residual moisture")

In [60]:
# Wpwp_FRACT MUST be >= resid_moist / (1.0 - bulk_density/soil_density).
if params['Wpwp_FRACT'].where(params['Wpwp_FRACT'] < params['resid_moist'] / porosity).sum() > 0:
    raise AssertionError("wilting point moisture is less than residual moisture")

__Save new parameters to NetCDF, location specified by `outdir` in config file__

In [61]:
encoding_params = {'run_cell': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'gridcell': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'fs_active': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'Nveg': {'dtype': 'int32', "_FillValue": fillval_i},
                   'overstory': {'dtype': 'int32', "_FillValue": fillval_i},
                   'veg_class': {'dtype': 'int32'}}

direc = config['Parameter Specs']['output_dir']

if organic_fract == True and bulk_density_comb == True and max_snow_albedo == True:
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "all_options")
elif organic_fract == False and bulk_density_comb == True and max_snow_albedo == False:
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "bulk_density")
else: 
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "no_options")
    
new_params_file = os.path.join(direc, filename)
params.to_netcdf(new_params_file, format='NETCDF4_CLASSIC', encoding=encoding_params)

if organic_fract:
    print("Organic Fract is True, including soil_dens_org and organic_fract in parameters file")
if max_snow_albedo:
    print("Max Snow Albedo is True, including veg-dep snow albedo in parameters file")
if bulk_density_comb:
    print("Bulk Density Comb is True, including combined bulk density in parameters file")
    
print("saved new parameters to %s" %new_params_file)

Organic Fract is True, including soil_dens_org and organic_fract in parameters file
Max Snow Albedo is True, including veg-dep snow albedo in parameters file
Bulk Density Comb is True, including combined bulk density in parameters file
saved new parameters to /p/home/gergel/data/parameters/25km/new_vic5_params_wr25b_ar9v4_all_options.nc
